In [1]:
#Importacion de librerias
#Libreria OpenCV
import cv2
#Libreria Numpy
import numpy as np
#Libreria de opciones de sistema
import os
#Dar importancia de mensajes
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import shutil
import glob
#Importar pandas
import pandas as pd
#Importar librerias graficas
import matplotlib.pyplot as plt
from  PIL  import Image
#Importar librerias de analisis
import seaborn as sn
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from rembg import remove
from sklearn.preprocessing import MinMaxScaler
import h5py
#Importar libreria de tiempo
from time import sleep

In [3]:
#Ruta de origen de .ipynb
root_path = os.path.abspath('.')
#Obtener ruta de carpeta con imagenes
images_path =root_path+"/Banana_Ripeness_Dataset_Augmented-main"
#Enlistar carpetas de imagenes
class_names = os.listdir(images_path)
#Clasificar carpetas
class_names.sort()
print(class_names)

['totally_green', 'yellow_with_brown_spots', 'yellow_with_green_tips']


In [ ]:
#copiar las bananas
#Ruta de imagnes aumentadas
banana_path= root_path+"/Banana_Ripeness_Dataset_Augmented"
#Iteracion por cada clase
for class_name in class_names:
    #Formacion de ruta de clase
    cur_path = images_path + "/" + class_name
    #Creacion de ruta de lementos
    elements = os.listdir(cur_path)
    #Creacion de ruta de archivos aumentados
    os.makedirs(banana_path+"/"+ class_name, exist_ok=True)
    
    for element in elements:
        print("Copying Image - {} in Banana_Ripeness_Dataset_Augmented/{}".format(element, class_name))
        src = os.path.join(images_path + "/" + class_name, element) # origen
        dst = os.path.join(banana_path+"/" + class_name, element) 
        shutil.copy(src, dst)


# Aumento de datos

In [ ]:
#Crear generador de datos de imagen
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.4,
        horizontal_flip=True,
        fill_mode='nearest')

# Clase totalmente verde

In [ ]:
#Cambiar de direccion a imagenes verde
os.chdir(banana_path+"/totally_green")
#Leer imagenes
for path in os.listdir():
    img = load_img(f"{path}")
    x = img_to_array(img)    # Esto es un arreglo Numpy array con dimension (3, 150, 150)
    x = x.reshape((1,) + x.shape)
    i = 0
    #Crear imagenes con formato jpg
    for batch in datagen.flow(x, batch_size=1,
            save_to_dir=".", 
            save_prefix= 'totally_green', 
            save_format='jpg'):
        i += 1
        if i == 1:     ## crear 10 imagenes de 1 imagen 
            break  

# Clase amarilla con puntas verdes

In [ ]:
#Cambiar de direccion a imagenes amarrillo y verde
os.chdir(banana_path+"/yellow_with_green_tips")
#Leer imagenes
for path in os.listdir():
    img = load_img(f"{path}")
    x = img_to_array(img)   # Esto es un arreglo Numpy array con dimension (3, 150, 150)
    x = x.reshape((1,) + x.shape)
    i = 0
    #Crear imagenes con formato jpg
    for batch in datagen.flow(x, batch_size=1,
            save_to_dir=".",  
            save_prefix= 'yellow_with_green_tips', 
            save_format='jpg'):
        i += 1
        if i == 9:     ## crear 10 imagenes de 1 imagen 
            break  

# Clase amarilla con manchas marrones

In [ ]:
#Cambiar de direccion a imagenes amarrillo y marron
os.chdir(banana_path+"/yellow_with_brown_spots")
#Leer imagenes
for path in os.listdir():
    img = load_img(f"{path}")
    x = img_to_array(img)    # Esto es un arreglo Numpy array con dimension (3, 150, 150)
    x = x.reshape((1,) + x.shape)
    i = 0
    #Crear imagenes con formato jpg    
    for batch in datagen.flow(x, batch_size=1,
            save_to_dir=".",   
            save_prefix= 'yellow_with_brown_spots', 
            save_format='jpg'):
        i += 1
        if i == 7:     ## crear 10 imagenes de 1 imagen 
            break 

# Filtro NLM para eliminación de ruido de imágenes

In [ ]:
os.chdir(root_path)
images_path = root_path+"/Banana_Ripeness_Dataset_Augmented"
class_names = os.listdir(images_path)
class_names.sort()
print(class_names)

In [ ]:
#Lectura de cada clase de imagen
for class_name in class_names:
    cur_path = images_path + "/" + class_name
    cur_label = class_name
    # Carpeta de modificados
    file_path = root_path+"/Banana_Ripeness_Dataset_Denoised/"+ cur_label
    #Creacion de rcarpeta para archivos modificaos
    os.makedirs(file_path, exist_ok=True)
    i = 1
    #Procesamiento de cada imeagen
    for file in glob.glob(cur_path + "/*"):
        print("Processing Image - {} in {}".format(i, cur_label))
        # lectura de ruta de imagen
        image = cv2.imread(file)
        # imagen filtrada
        dst = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
        b,g,r = cv2.split(dst)
        rgb_dst = cv2.merge([r,g,b])
        # guardar imagen
        save_path = file_path
        cv2.imwrite(os.path.join(save_path , cur_label + '_' + str(i) + '.jpg'),
                    cv2.cvtColor(rgb_dst,cv2.COLOR_RGB2BGR))
        # conteo de procesos
        i += 1


# Eliminación de fondo

In [4]:
#Ruta de imagenes filtradas
images_path = root_path+"/Banana_Ripeness_Dataset_Denoised/"
#BUsqueda de clases
class_names = os.listdir(images_path)
class_names.sort()
#Impresion de clases
print(class_names)

['totally_green', 'yellow_with_brown_spots', 'yellow_with_green_tips']


In [ ]:
#Iteracion en cada clase
for class_name in class_names:
    #Ruta de clase
    cur_path = images_path + "/" + class_name
    #Nombre de calse
    cur_label = class_name
    # Carpeta de modificados
    file_path = root_path+"/Banana_Ripeness_Dataset_RemovedBG/"+ cur_label
    #Creacion de rcarpeta para archivos modificaos
    os.makedirs(file_path, exist_ok=True)
    i = 1
    #Procesamiento de cada imagen
    for file in glob.glob(cur_path + "/*"):
        print("Processing Image - {} in {}".format(i, cur_label))
        # Lectura de imagen
        image = cv2.imread(file)
        # remocion de fondo
        output = remove(image)
        # GUardado de imagen
        save_path = file_path
        cv2.imwrite(os.path.join(save_path , cur_label + '_' + str(i) + '.png'), output)
        # Conteo de proceso
        i += 1

Processing Image - 1 in totally_green


Downloading...
From: https://drive.google.com/uc?id=1tCU5MM1LhRgGou5OpmpjBQbSrYIUoYab
To: C:\Users\FABIOLA\.u2net\u2net.onnx
100%|████████████████████████████████████████████████████████████████████████████████| 176M/176M [03:15<00:00, 899kB/s]


Processing Image - 2 in totally_green
Processing Image - 3 in totally_green
Processing Image - 4 in totally_green
Processing Image - 5 in totally_green
Processing Image - 6 in totally_green
Processing Image - 7 in totally_green
Processing Image - 8 in totally_green
Processing Image - 9 in totally_green
Processing Image - 10 in totally_green
Processing Image - 11 in totally_green
Processing Image - 12 in totally_green
Processing Image - 13 in totally_green
Processing Image - 14 in totally_green
Processing Image - 15 in totally_green
Processing Image - 16 in totally_green
Processing Image - 17 in totally_green
Processing Image - 18 in totally_green
Processing Image - 19 in totally_green
Processing Image - 20 in totally_green
Processing Image - 21 in totally_green
Processing Image - 22 in totally_green
Processing Image - 23 in totally_green
Processing Image - 24 in totally_green
Processing Image - 25 in totally_green
Processing Image - 26 in totally_green
Processing Image - 27 in totally_